In the first notebook, a table was created that described the distance between points every 50 meters along each modeling network segment. This table will be provide the basis of the conflation between the two networks. The process will start with divided highways and then the lower class segments. This process uses the pandas module, and a variety of sorting methods to determine 'good' matches.

In [1]:
# pandas will take care of most of the sorting and filtering
import pandas as pd

# the arcpy module will be used to create lines between the filtered matches so that these matches can be checked visually
import arcpy as ap

# os is used to determine the project location
import os
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
path_to_data = os.getcwd() + '\\Data\\'
dtd_network_file = os.getcwd() + '\\CDOT_Network_2015_HwyLinks.shp'

The function below is used to calculate the difference between bearings, this is a primary comparison between the networks

In [2]:
def getDifference(b1, b2):
    r = (b2 - b1) % 360.0
    # Python modulus has same sign as divisor, which is positive here,
    # so no need to consider negative case
    if r >= 180.0:
        r -= 360.0
    return r

The cell below loads the csv file for each network's attributes and the match file. The attributes are then joined to the match file.

In [3]:
# store the matches .csv as a pandas dataframe
matches = pd.read_csv(path_to_data + 'matches.csv')

# store each of the attribute tables as a pandas dataframe
tmc_attributes = pd.read_csv(path_to_data + 'tmc_attributes.csv')
dtd_attributes = pd.read_csv(path_to_data + 'dtd_attributes.csv')

# join each attribute dataframe to the matches dataframe
matches_with_tmc_attributes = pd.merge(matches, tmc_attributes, on = 'tmc_id')
matches_with_attributes = pd.merge(matches_with_tmc_attributes, dtd_attributes, on = 'dtd_id')

#del matches, tmc_attributes, dtd_attributes, matches_with_tmc_attributes

matches_with_attributes.head()

,match_id,tmc_id,dtd_id,distance,NEAR_RANK,NEAR_ANGLE,FROM_X,FROM_Y,NEAR_X,NEAR_Y,tmc_name,tmc_bearing,TMC,tmc_direction,tmc_type,dtd_name,dtd_type,dtd_bearing,dtd_one_way
0,0,0,21694,1.184238,1,158.271560,-105.518645,40.377374,-105.518640,40.377364,E Elkhorn Ave,256.21,116+07047,+,2,US-34,3,252.9,Y
1,1,0,21694,3.861466,1,158.271198,-105.519205,40.377229,-105.519189,40.377197,E Elkhorn Ave,256.21,116+07047,+,2,US-34,3,252.9,Y
2,2,0,21694,5.436999,1,169.300070,-105.519765,40.377081,-105.519753,40.377033,E Elkhorn Ave,256.21,116+07047,+,2,US-34,3,252.9,Y
3,5,0,21694,1.896267,1,-10.700291,-105.520324,40.376933,-105.520328,40.376950,E Elkhorn Ave,256.21,116+07047,+,2,US-34,3,252.9,Y
4,10,0,21694,30.492439,2,61.826387,-105.520884,40.376785,-105.520567,40.376915,E Elkhorn Ave,256.21,116+07047,+,2,US-34,3,252.9,Y


In [4]:
# filter the modeling highways from all of the matches
highway_matches_with_attributes = matches_with_attributes[matches_with_attributes['dtd_type'] == 1]

# filter the tmc matches from the modeling highways dataframe
highway_matches_with_attributes = highway_matches_with_attributes[
                                    highway_matches_with_attributes['tmc_type'] == 1]

# calculate the difference in bearing between all of the highway matches
highway_matches_with_attributes['angle_diff'] = highway_matches_with_attributes.apply(
                                                          lambda x: 
                                                          abs(getDifference(x['tmc_bearing'], 
                                                                            x['dtd_bearing'])), 
                                                          axis=1
                                                          )

# if the difference in bearing is less than 45 degrees, keep the matches
highway_matches_with_attributes = highway_matches_with_attributes[highway_matches_with_attributes['angle_diff'] < 45]

# sort the matches by each tmc point's X and Y 
highway_matches_with_attributes = highway_matches_with_attributes.sort_values(['FROM_X', 'FROM_Y', 'distance'])

# eliminate all of the matches that aren't the closest 
closest_highway_matches = highway_matches_with_attributes.drop_duplicates(['FROM_X', 'FROM_Y'])

# clean up
del highway_matches_with_attributes

closest_highway_matches.head()

,match_id,tmc_id,dtd_id,distance,NEAR_RANK,NEAR_ANGLE,FROM_X,FROM_Y,NEAR_X,NEAR_Y,tmc_name,tmc_bearing,TMC,tmc_direction,tmc_type,dtd_name,dtd_type,dtd_bearing,dtd_one_way,angle_diff
221520,191755,3145,7117,19.754610,1,-171.225184,-109.051246,39.194227,-109.051281,39.194051,I-70 W,290.11,116-04864,-,1,070A,1,277.78,N,12.33
221567,421500,6381,7117,15.714377,1,-171.225055,-109.051041,39.194166,-109.051069,39.194026,I-70 W,278.52,116-04679,-,1,070A,1,277.78,N,0.74
318633,398610,6022,1934,2.976128,1,40.131662,-109.050745,39.193095,-109.050723,39.193115,I-70 E,98.83,116+04680,+,1,070A,1,100.15,N,1.32
221519,191754,3145,7117,8.842645,1,-171.224836,-109.050692,39.194061,-109.050708,39.193982,I-70 W,290.11,116-04864,-,1,070A,1,277.78,N,12.33
221566,421499,6381,7117,4.802531,1,-171.224706,-109.050487,39.194000,-109.050495,39.193957,I-70 W,278.52,116-04679,-,1,070A,1,277.78,N,0.74


In [ ]:
# write the resulting dataframe to a .csv file
closest_highway_matches.to_csv(path_to_data + 'highway_matches.csv')

# and convert the .csv file of lines to a line feature class
ap.XYToLine_management(path_to_data + 'highway_matches.csv', 
                         path_to_data + 'highway_match_lines.shp',
                         'FROM_X',
                         'FROM_Y',
                         'NEAR_X',
                         'NEAR_Y',
                         'GEODESIC')

In [5]:
# filter the not modeling highways from all of the matches
not_highway_matches_with_attributes = matches_with_all_attributes[matches_with_all_attributes['dtd_type'] > 1]

# filter the not tmc matches from the modeling highways dataframe
not_highway_matches_with_attributes = not_highway_matches_with_attributes[not_highway_matches_with_attributes['tmc_type'] == 2]


not_highway_matches_with_attributes['angle_diff'] = not_highway_matches_with_attributes.apply(
                                                          lambda x: 
                                                          abs(getDifference(x['tmc_bearing'], 
                                                                            x['dtd_bearing'])), 
                                                          axis=1
                                                          )
# if the difference in bearing is less than 45 degrees, store the matches to a new dataframe
not_highway_matches_with_attributes_1 = not_highway_matches_with_attributes[
                                        not_highway_matches_with_attributes['angle_diff'] < 45]

# if the difference in bearing is more than 135 degrees, store the matches to a new dataframe
not_highway_matches_with_attributes_2 = not_highway_matches_with_attributes[
                                        not_highway_matches_with_attributes['angle_diff'] > 135]

# append the two dataframes created above and store to the not_highway_matches_with_attributes dataframe
not_highway_matches_with_attributes = not_highway_matches_with_attributes_1.append(not_highway_matches_with_attributes_2)

# clean up the previously created data frames
del not_highway_matches_with_attributes_1, not_highway_matches_with_attributes_2

# calculate the difference between the bearing of the line between near points and the bearing of the tmc segment
not_highway_matches_with_attributes['near_angle_diff'] = not_highway_matches_with_attributes.apply(
                                                          lambda x: 
                                                          abs(getDifference(x['tmc_bearing'], 
                                                                            x['NEAR_ANGLE'])), 
                                                          axis=1
                                                          )
# remove all the matches athat are over 20 Meters
not_highway_matches_with_attributes = not_highway_matches_with_attributes[
                                      not_highway_matches_with_attributes['distance'] < 20]

# sort all the matches by each point along the tmc line segments
not_highway_matches_with_attributes = not_highway_matches_with_attributes.sort_values(['FROM_X', 'FROM_Y', 'distance'])

# remove all of the matches that aren't perpindicular to the tmc network segment
not_highway_matches_with_attributes = not_highway_matches_with_attributes.query('near_angle_diff < 110')
not_highway_matches_with_attributes = not_highway_matches_with_attributes.query('near_angle_diff > 70')

# remove all of the matches that aren't the closest
closest_not_highway_matches = not_highway_matches_with_attributes.drop_duplicates(['FROM_X', 'FROM_Y'])

# clean up
del not_highway_matches_with_attributes

closest_not_highway_matches.head()

In [ ]:
# save the closest_not_highway_matches dataframe to a csv file
closest_not_highway_matches.to_csv(path_to_data + 'not_highway_matches.csv')

# convert the not_highway_matches csv file to an arc shapeline 
ap.XYToLine_management(path_to_data + 'not_highway_matches.csv', 
                         path_to_data + 'not_highway_match_lines.shp',
                         'FROM_X',
                         'FROM_Y',
                         'NEAR_X',
                         'NEAR_Y',
                         'GEODESIC')

The cell below will start seperating the tmcs into a positive and negative direction, most of our modeling network consists of bidirectional segments, so these will have both a positive and negative tmc assigned to them. First the negative and positives will be split into seperate dataframes

In [6]:
# append the not_highway_matches and highway_matches dataframes to one al_matches_dataframe
all_matches = closest_not_highway_matches.append(closest_highway_matches)
all_matches = all_matches.filter(['dtd_id', 'TMC', 'tmc_direction'])

# seperate the negative and positive tmcs into their own dataframe
n_matches = all_matches[all_matches['tmc_direction'].isin(['N', '-'])]
p_matches = all_matches[all_matches['tmc_direction'].isin(['P', '+'])]

# clean up
del all_matches, closest_not_highway_matches, closest_highway_matches

In [9]:
# group the n_matches dataframe by the dtd_id and TMC and count how many of each match there are 
n_matches = n_matches.groupby(['dtd_id', 'TMC']).agg('count')

# save the dataframe to a csv file, essentially flattening the file (this should be tightened up with reindexing)
n_matches.to_csv(path_to_data + 'n_matches.csv')

# open the csv created above back into a dataframe
n_matches = pd.read_csv(path_to_data + 'n_matches.csv')

# rename the columns of the dataframe for clarity
n_matches.columns = ['dtd_id', 'TMC', 'count']

# sort the matched columns according to the count
n_matches = n_matches.sort_values(['dtd_id','count'])

# remove any matches that don't have the highest count
n_matches = n_matches.drop_duplicates(['dtd_id'], 'last')

# clean up the dataframe to only include the dtd_id and tmc
n_matches = n_matches.filter(['dtd_id', 'TMC'])

#rename the columns for clarity
n_matches.columns = ['dtd_id', 'N_TMC']
n_matches.head()

,dtd_id,N_TMC
0,3,116-05346
1,4,116-06669
4,6,116-05991
5,12,116-06078
6,15,116-05615


In [10]:
p_matches = p_matches.groupby(['dtd_id', 'TMC']).agg('count')

p_matches.to_csv(path_to_data + 'p_matches.csv')

p_matches = pd.read_csv(path_to_data + 'p_matches.csv')

p_matches.columns = ['dtd_id', 'TMC', 'count']

p_matches = p_matches.sort_values(['dtd_id','count'])

p_matches = p_matches.drop_duplicates(['dtd_id'], 'last')
p_matches = p_matches.filter(['dtd_id', 'TMC'])

p_matches.columns = ['dtd_id', 'P_TMC']
p_matches.head()

,dtd_id,P_TMC
0,2,116+04694
1,3,116+05347
4,4,116+06853
5,6,116+05992
6,10,116+04629


In [11]:
dtd_ids = pd.read_csv(path_to_data + 'dtd_attributes.csv')
dtd_ids = dtd_ids.filter(['dtd_id'])

dtd_ids.columns = ['dtd_id']
dtd_ids.head()

,dtd_id
0,0
1,1
2,2
3,3
4,4


In [12]:
# join the negative and positive tmcs to the dtd_id
dtd_ids_with_n_matches = pd.merge(dtd_ids, n_matches, how = 'left')
dtd_ids_with_n_and_p_matches = pd.merge(dtd_ids_with_n_matches, p_matches, how = 'left')

del dtd_ids, dtd_ids_with_n_matches

dtd_ids_with_n_and_p_matches.head()

,dtd_id,N_TMC,P_TMC
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,116+04694
3,3,116-05346,116+05347
4,4,116-06669,116+06853


In [13]:
# save the results to a csv file so they can be joined to any file with the associated zone_id
dtd_ids_with_n_and_p_matches.to_csv(path_to_data + 'dtd_id_to_TMC_link_table.csv')